# Preprocess

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vinceflores/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load DAtasets

In [3]:
pd.set_option('max_colwidth', 800)

TWEETS_PATH= "../data/raw_tweets_text.csv"
SENTIMENT_PATH="../data/t4sa_text_sentiment.tsv"

#load data
tweets_df = pd.read_csv(TWEETS_PATH, encoding='latin-1', header=0)
sentiment_df= pd.read_csv(SENTIMENT_PATH, sep='\t', header=0)

In [4]:
# Useful functions to help extract data from the columns

def extract_username_from_text(text):     # Extracts the username from retweets (e.g., "RT @user:").
    match = re.search(r'^RT @([^\s:]+):', text)
    return match.group(1) if match else None
     
def extract_links_from_text(text): # Extracts URLs from tweet text.
    urls = re.findall(r'https?://\S+', text)
    if not urls:
        return None
    return urls[0] if len(urls) == 1 else urls
           
def extract_hashtags_from_text(text): # Extract hashtags from the tweet text
    hashtags = re.findall(r'#\w+', text)
    if not hashtags:
        return None
    return hashtags[0] if len(hashtags) == 1 else hashtags
    
def extract_mentions_from_text(text): # Extract mentions from the tweet text
    cleaned_text = re.sub(r'^RT @[^\s:]+: ', '', text) # Remove the initial retweet username (e.g., "RT @user:")
    mentions = re.findall(r'@\w+', cleaned_text)
    if not mentions:
        return None
    return mentions[0] if len(mentions) == 1 else mentions



def clean_tweet_text(text: str):
    """
    Cleans the tweet text for EDA by removing noise such as:
    - Retweet prefixes (RT @user:)
    - URLs
    - HTML entities (e.g., &amp;)
    - Extra whitespace
    - Remove mentions
    - remove # symbol
    """
    # Remove retweet header
    text = re.sub(r'^RT @[^\s:]+: ', '', text)
    
    # Remove URLs
    # Remove URLs including malformed/truncated ones (e.g., "httpsâ")
    text = re.sub(r'https?\S+|www\.\S+', '', text)
    
    # Remove HTML entities like &amp;
    text = re.sub(r'&\w+;', '', text)
    
    # Remove extra spaces and trim
    text = re.sub(r'\s+', ' ', text).strip()

    #remove metions
    text = re.sub(r'@\w+', '', text)
    # remove hashtags
    # text = re.sub(r'#', '\w', text)
    text = text.replace("#","", -1)
    
    return text

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
# nltk.download('stowords')
def rm_stop_words(tweet):
    tokenized_tw = word_tokenize(tweet)
    stop_words = set(stopwords.words('english'))
    result = [t for t in tokenized_tw if t.lower() not in stop_words ]
    return " ".join(result)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/vinceflores/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
merged_df = pd.merge(tweets_df, sentiment_df, left_on='id', right_on='TWID')
merged_df = merged_df.drop(columns=['TWID']) # since its alr in id

main_df = merged_df.copy()
# Start adding extra columns that might help us with visualizations
# main_df['is_retweet'] = main_df['text'].str.startswith('RT ')
# main_df['username'] = main_df['text'].apply(extract_username_from_text)
# # main_df['urls'] = main_df['text'].apply(extract_links_from_text)
# main_df['hashtags'] = main_df['text'].apply(extract_hashtags_from_text)
# main_df['mentions'] = main_df['text'].apply(extract_mentions_from_text)

main_df['text'] = main_df['text'].apply(clean_tweet_text)
main_df['text'] = main_df['text'].apply(rm_stop_words)
classes=['NEG', 'POS', 'NEU']
main_df['class'] = main_df[classes].idxmax(axis=1)
cols_to_drop = ['id', 'NEG', 'POS', 'NEU'  ]
main_df = main_df.drop(columns=cols_to_drop, axis=1)
main_df


,text,class
0,Josh Jenkins looking forward TAB Breeders Crown Super Sunday,POS
1,[ Pic ] Nichkhun krjeong86 's IG,NEU
2,Congratulations Pakistan becoming No1TestTeam world odds ! JI_PakZindabadRallies,POS
3,"September , taking Maine Mendozaâs surprise thanksgiving party threw fans !",POS
4,Incredible India Atulya Bharat - Land Seekers BeProud ð ð®ð³ : | : Plz RT,NEU
...,...,...
1179952,morning girls wonderful Friday,POS
1179953,RT Follow Colin Kaepernick debated merits Castro'sâ¦ - Mercury News,NEU
1179954,live webcam find download app,NEU
1179955,Pearl Roadshow 4-piece Complete Drum Set Cymb,NEU


In [7]:
saved_path = "../data/processed_tweets_sentiment.csv"
main_df.to_csv(saved_path, index=False)

In [8]:
!pip install sentence-transformers

In [9]:
from sentence_transformers import SentenceTransformer
# Initialize the SBERT model
sbert = SentenceTransformer('bert-base-nli-mean-tokens')
sample_size = 5000
random_state = 33
neu = main_df[main_df['class'] == 'NEU'].sample(n=sample_size, random_state=random_state)
neg = main_df[main_df['class'] == 'NEG'].sample(n=sample_size, random_state=random_state)
pos = main_df[main_df['class'] == 'POS'].sample(n=sample_size, random_state=random_state)
# subset = main_df.head(1000)
subset = pd.concat([neu,neg, pos], ignore_index=True)

# tweet_embeddings = sbert.encode(subset['text'])


In [10]:
# subset['embeddings'] = tweet_embeddings.tolist()


In [11]:
# save_path="../project_data/t4sa_data.csv"
# subset.to_csv(save_path, index=False)

# Embedding Using Word2vec

In [12]:
! pip install gensim

In [13]:
dataset = main_df.loc[main_df['class'].isin(['NEG', 'NEU', 'POS'])]
dataset.columns

Index(['text', 'class'], dtype='object')

In [14]:

# tokenized_tw = word_tokenize(tweet)
dataset["tokenized_text"] = dataset["text"].apply(lambda x: word_tokenize(x))

In [15]:
# dataset["tokenized_text"]
dataset

,text,class,tokenized_text
0,Josh Jenkins looking forward TAB Breeders Crown Super Sunday,POS,"[Josh, Jenkins, looking, forward, TAB, Breeders, Crown, Super, Sunday]"
1,[ Pic ] Nichkhun krjeong86 's IG,NEU,"[[, Pic, ], Nichkhun, krjeong86, 's, IG]"
2,Congratulations Pakistan becoming No1TestTeam world odds ! JI_PakZindabadRallies,POS,"[Congratulations, Pakistan, becoming, No1TestTeam, world, odds, !, JI_PakZindabadRallies]"
3,"September , taking Maine Mendozaâs surprise thanksgiving party threw fans !",POS,"[September, ,, taking, Maine, Mendozaâs, surprise, thanksgiving, party, threw, fans, !]"
4,Incredible India Atulya Bharat - Land Seekers BeProud ð ð®ð³ : | : Plz RT,NEU,"[Incredible, India, Atulya, Bharat, -, Land, Seekers, BeProud, ð, ð®ð³, :, |, :, Plz, RT]"
...,...,...,...
1179952,morning girls wonderful Friday,POS,"[morning, girls, wonderful, Friday]"
1179953,RT Follow Colin Kaepernick debated merits Castro'sâ¦ - Mercury News,NEU,"[RT, Follow, Colin, Kaepernick, debated, merits, Castro'sâ¦, -, Mercury, News]"
1179954,live webcam find download app,NEU,"[live, webcam, find, download, app]"
1179955,Pearl Roadshow 4-piece Complete Drum Set Cymb,NEU,"[Pearl, Roadshow, 4-piece, Complete, Drum, Set, Cymb]"


In [16]:
# reference https://radimrehurek.com/gensim/models/word2vec.html
from gensim.test.utils import lee_corpus_list
from gensim.models import Word2Vec
sentences = dataset['tokenized_text']

vector_size=10
window=5
min_count=1
workers=4

w2v = Word2Vec(sentences,vector_size=vector_size, window=window, min_count=min_count, workers=workers, sg=1)

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_fl

In [17]:
def create_w2v_embeddings(row, model):
    e = []
    for token in row:
        # e.append(np.mean( model.wv[token]))
        # e.append(np.mean())
        e.append( model.wv[token])
    return e
dataset['embeddings'] = dataset["tokenized_text"].apply(lambda row: create_w2v_embeddings(row, w2v))


In [18]:

dataset['embeddings_flatten'] = dataset["embeddings"].apply(lambda row: np.array(row).reshape(-1).tolist())

In [19]:
dataset.head()

,text,class,tokenized_text,embeddings,embeddings_flatten
0,Josh Jenkins looking forward TAB Breeders Crown Super Sunday,POS,"[Josh, Jenkins, looking, forward, TAB, Breeders, Crown, Super, Sunday]","[[-0.113853954, 0.61426413, 1.0451182, 0.893222, 1.2579755, 0.35145238, 1.4110141, 1.1207917, -1.3057975, -0.72572553], [-0.35403937, 1.0621666, 0.74526286, 0.7480261, 1.2924874, 0.7369891, 1.1482413, 0.80118775, -1.3500674, -0.44346103], [-0.2957207, -0.53411484, 0.6827601, -0.28330567, 1.731146, -0.55176055, 1.518203, 0.25636676, -1.2183276, -1.5224853], [0.051040474, -0.7260078, -0.1618401, 0.46670628, 1.2540244, -0.9407606, 2.0366619, 0.47104907, -1.8512024, -2.0404606], [-1.2794154, 0.39127317, 0.14349873, -0.98931277, 0.0033254607, 1.4170622, 0.5298591, 1.460339, -0.6791693, -1.0214216], [-0.25402245, -0.0016779394, 0.08299904, -0.16346505, 0.30869868, 0.54760236, 0.59448117, 0.46310663, -0.84121794, -0.5000733], [-0.16067526, 0.60375583, 0.3016045, -0.6323253, 0.99004763, 0.8514...","[-0.11385395377874374, 0.6142641305923462, 1.0451182126998901, 0.8932219743728638, 1.257975459098816, 0.35145238041877747, 1.4110140800476074, 1.1207916736602783, -1.3057974576950073, -0.725725531578064, -0.35403937101364136, 1.0621665716171265, 0.745262861251831, 0.7480260729789734, 1.292487382888794, 0.7369890809059143, 1.1482412815093994, 0.8011877536773682, -1.350067377090454, -0.44346103072166443, -0.2957206964492798, -0.5341148376464844, 0.6827601194381714, -0.28330567479133606, 1.731145977973938, -0.5517605543136597, 1.5182030200958252, 0.2563667595386505, -1.2183276414871216, -1.5224852561950684, 0.05104047432541847, -0.7260078191757202, -0.16184009611606598, 0.4667062759399414, 1.2540243864059448, -0.940760612487793, 2.0366618633270264, 0.47104907035827637, -1.851202368736267,..."
1,[ Pic ] Nichkhun krjeong86 's IG,NEU,"[[, Pic, ], Nichkhun, krjeong86, 's, IG]","[[-1.5693468, 2.049658, 1.137492, 0.08750274, 0.54676276, 0.63230544, 0.6729003, 0.655981, -1.3121158, -1.4043839], [-0.4361653, 1.1729405, 0.8765881, 0.028632604, 0.64229333, 0.8665044, 1.1770998, 0.4567495, -0.93769294, -0.7557075], [-1.6565623, 2.1175146, 1.2094496, 0.18033363, 0.53936005, 0.6481038, 0.6693481, 0.6759012, -1.2341661, -1.5299729], [-1.5265535, 1.191525, 0.9147903, 0.7891117, 0.8964215, 1.0092945, 0.59716886, 0.4279841, -0.52998984, -1.2941233], [-0.16759256, 0.009424157, -0.011900795, 0.041153755, 0.16186744, 0.25351366, -0.008999251, -0.025426328, -0.048780646, -0.15624174], [-0.7074431, 0.23060036, 1.0581964, -0.19098715, 1.2275993, -0.31913444, 1.7760433, 0.6697373, -1.1851416, -0.5484665], [-1.1426878, 1.2944506, 1.9087994, 0.3696698, 1.1195431, 0.5566722, 0.6645...","[-1.5693467855453491, 2.0496580600738525, 1.1374919414520264, 0.08750274032354355, 0.5467627644538879, 0.6323054432868958, 0.6729003190994263, 0.6559810042381287, -1.3121157884597778, -1.404383897781372, -0.43616530299186707, 1.1729404926300049, 0.8765881061553955, 0.028632603585720062, 0.6422933340072632, 0.8665043711662292, 1.1770998239517212, 0.45674949884414673, -0.9376929402351379, -0.7557075023651123, -1.656562328338623, 2.1175146102905273, 1.2094496488571167, 0.18033362925052643, 0.5393600463867188, 0.6481037735939026, 0.6693481206893921, 0.6759011745452881, -1.234166145324707, -1.5299729108810425, -1.5265535116195679, 1.1915249824523926, 0.9147902727127075, 0.7891116738319397, 0.896421492099762, 1.0092945098876953, 0.5971688628196716, 0.4279840886592865, -0.5299898386001587, -1..."
2,Congratulations Pakistan becoming No1TestTeam world odds ! JI_PakZindabadRallies,POS,"[Congratulations, Pakistan, becoming, No1TestTeam, world, odds, !, JI_PakZindabadRallies]","[[0.1317918, -0.14983346, 0.8323126, 0.60160947, 0.096248366, 0.20120004, 1.9938416, 0.5724737, -1.4372393, -1.0748318], [-0.61821544, -0.9514865, 1.4258574, 1.1346506, 0.83760864, -0.30233258, 0.6428912, 0.78054726, -2.247871, -0.23723312], [-0.5618646, -0.24005282, 0.8093832, 0.570356, 1.4414698, -0.87038577, 1.4082448, 0.7151696

In [21]:

save_path="../project_data/t4sa_data_w2v_sg.csv"
subset.to_csv(save_path, index=False)